In [ ]:
!pip install wandb huggingface_hub transformers datasets accelerate bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
import wandb
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import pipeline
import os

In [ ]:
# Login to Weights and Biases and Hugging Face Hub
wandb.login()
notebook_login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Set seed for reproducibility
torch.manual_seed(42)

# Load dataset
dataset = load_dataset("ArtifactAI/arxiv_python_research_code")
dataset_train = dataset["train"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1415924 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
# Prepare datasets for training and validation
split_datasets = DatasetDict({
    "train": dataset_train.shuffle(seed=42).select(range(50000)),
    "valid": dataset_train.shuffle(seed=42).select(range(500))
})

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")
context_length = 256

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
# Tokenization function
def tokenize_data(batch):
    outputs = tokenizer(
        batch["code"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True
    )
    batch_input_ids = [
        input_ids for length, input_ids in zip(outputs["length"], outputs["input_ids"]) if length == context_length
    ]
    return {"input_ids": batch_input_ids}

# Tokenize datasets
tokenized_datasets = split_datasets.map(tokenize_data, batched=True, remove_columns=split_datasets["train"].column_names)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Configure model
model_config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# Initialize model
model = GPT2LMHeadModel(model_config)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Prepare data collator
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Quack/python_code_gen",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    logging_steps=25,
    eval_steps=25,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    learning_rate=1e-3,
    save_steps=100,
    fp16=True,
    report_to="wandb"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"]
)

In [ ]:
# Fine-tune the model
trainer.train()
trainer.push_to_hub()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
0,2.115700,2.183318


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/phoenix28/python_code_gen/commit/0ddfc5ec988287b35b70cda0b74a6c95612f9294', commit_message='End of training', commit_description='', oid='0ddfc5ec988287b35b70cda0b74a6c95612f9294', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Integrate quantization
from transformers import GPT2LMHeadModel
#from torch.quantization import quantize_dynamic

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Quack/python_code_gen")
#quantized_model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

In [ ]:
# Initialize pipeline for inference
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
generator = pipeline(
    "text-generation", model='phoenix28/python_code_gen', tokenizer=tokenizer, device=device
)

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
wandb.finish()

In [ ]:
# Function to generate output
def generate_output(prompt):
    return generator(prompt, num_return_sequences=1)[0]["generated_text"]

In [ ]:
# Example prompt
prompt = """
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""
print(generate_output(prompt))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
y = np.linspace(-1.0, 1.


In [ ]:
print(generate_output(prompt))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
x = np.random.randn(100)
x
